In [1]:
import pycaret
from pycaret.regression import *
from pycaret.classification import *
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from tableone import TableOne

In [2]:
post = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\FDATA_Post_2B_GAUS_NORM_RESAMPLE000_5Sig_2023-12-17-19-53-59_radiomics_features_ALL.csv", index_col=0) 
post.set_index('pt_no', inplace=True)
current_post = post.columns.tolist()
new_post = {col: 'post_' + col for col in current_post}
post = post.rename(columns=new_post)
post

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,post_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,post_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_ZoneEntropy,post_wavelet-LLL_glszm_ZonePercentage,post_wavelet-LLL_glszm_ZoneVariance,post_wavelet-LLL_ngtdm_Busyness,post_wavelet-LLL_ngtdm_Coarseness,post_wavelet-LLL_ngtdm_Complexity,post_wavelet-LLL_ngtdm_Contrast,post_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,2.212185,0.035648,3.621928,0.003134,1.429334e+06,15.156426,0.003068,0.932926,0.002431,0.037266
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,2.399476,0.012666,5.709832,0.003814,5.948491e+05,14.851495,0.002102,1.266959,0.005343,0.028783
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,3.027236,0.200319,4.477880,0.000856,1.198165e+07,74.174093,0.000500,1.435507,0.010100,0.008668
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,6.880579,0.013353,3.509561,0.001170,1.839469e+07,71.012121,0.000532,1.616311,0.008991,0.006797
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,5.894270,0.043460,3.628726,0.002132,2.952561e+06,148.316482,0.000469,3.045954,0.013363,0.004109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,0.490065,0.490065,2.404010,0.001197,6.828967e+06,12.033645,0.021085,0.010323,0.000106,0.019846
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,4.318118,0.093450,4.398636,0.001724,2.190314e+07,92.347019,0.000635,0.615894,0.001000,0.004040
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,6.316938,0.040322,4.143625,0.000649,6.753192e+07,40.683255,0.000882,0.522524,0.002792,0.009614


In [3]:
ypCR_data = pd.read_excel(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\clinical_immunologic.xlsx", index_col=0) 
ypCR_data.set_index('pt_no', inplace=True)
ypCR_data = ypCR_data[ypCR_data['Pathol_available'] == 1]
ypCR_data

,name,Ineligible,Ineligible_2019,Pathol_available,Hospital,note,base.fu,date.dx,dob,age,...,note.1,other.malig,pre_PDL1_CPS,pre_PDL1_IC,pre_CD3_num_Pos_mm,pre_CD8_num_Pos_mm,post_PDL1_CPS,post_PDL1_IC,post_CD3_num_Pos_mm,post_CD8_num_Pos_mm
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,정종일,0,0,1.0,1,NaN,2016-11-17,2016-10-24,1946-03-05,69.731507,...,NaN,NaN,5.0,5.0,366.0532,51.9522,10.0,10.0,3171.7243,890.4297
17815426,김진영,1,0,1.0,1,NaN,2021-07-30,2021-07-08,1955-01-04,65.660274,...,NaN,NaN,50.0,40.0,637.6734,374.4912,80.0,80.0,1532.5386,1298.4060
20109646,서대현,0,0,1.0,1,NaN,2018-07-09,2018-05-18,1951-06-20,66.134247,...,NaN,NaN,0.5,0.5,60.6479,39.7833,0.0,0.0,100.5308,43.6339
22320836,최흥우,0,0,1.0,1,NaN,2016-06-07,2016-05-18,1947-07-11,67.961644,...,"diaphragmatic hernia reduction (2020/3/17, 201...",NaN,10.0,10.0,608.9312,140.8317,10.0,10.0,1176.5166,1516.6479
32221501,박남훈,0,0,1.0,1,NaN,2010-10-11,2010-09-17,1939-10-16,70.013699,...,NaN,NaN,1.0,1.0,504.0535,64.6015,10.0,10.0,1245.4099,1155.8683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,최정옥,1,0,1.0,1,Previous malignancy: Breast ca (하지만 1995년이라 ok),2021-10-26,2021-09-29,1961-11-27,59.093151,...,NaN,NaN,10.0,5.0,1152.7963,471.6934,100.0,70.0,1477.2669,868.1072
56124435,김정자,1,0,1.0,1,NaN,2021-12-03,2021-11-09,1963-01-13,58.082192,...,NaN,NaN,0.0,0.0,145.0536,31.3657,10.0,10.0,479.3238,389.0902
56131318,홍성배,1,0,1.0,1,NaN,2021-11-23,2021-11-10,1951-05-20,69.542466,...,NaN,NaN,10.0,10.0,5283.4120,5773.6300,50.0,50.0,4583.2514,4448.2354


In [4]:
post['post_PDL1_CPS'] = ypCR_data['post_PDL1_CPS']
post

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,post_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_ZoneEntropy,post_wavelet-LLL_glszm_ZonePercentage,post_wavelet-LLL_glszm_ZoneVariance,post_wavelet-LLL_ngtdm_Busyness,post_wavelet-LLL_ngtdm_Coarseness,post_wavelet-LLL_ngtdm_Complexity,post_wavelet-LLL_ngtdm_Contrast,post_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,0.035648,3.621928,0.003134,1.429334e+06,15.156426,0.003068,0.932926,0.002431,0.037266,10.0
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,0.012666,5.709832,0.003814,5.948491e+05,14.851495,0.002102,1.266959,0.005343,0.028783,80.0
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,0.200319,4.477880,0.000856,1.198165e+07,74.174093,0.000500,1.435507,0.010100,0.008668,0.0
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,0.013353,3.509561,0.001170,1.839469e+07,71.012121,0.000532,1.616311,0.008991,0.006797,10.0
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,0.043460,3.628726,0.002132,2.952561e+06,148.316482,0.000469,3.045954,0.013363,0.004109,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,0.490065,2.404010,0.001197,6.828967e+06,12.033645,0.021085,0.010323,0.000106,0.019846,100.0
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,0.093450,4.398636,0.001724,2.190314e+07,92.347019,0.000635,0.615894,0.001000,0.004040,10.0
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,0.040322,4.143625,0.000649,6.753192e+07,40.683255,0.000882,0.522524,0.002792,0.009614,50.0


In [5]:
# bin_labels = [0, 1, 2]
# bins = [-1, 1, 10, float('inf')]
# post['encoded_PDL1'] = pd.cut(post['post_PDL1_CPS'], bins=bins, labels=bin_labels)
# post
conditions = [
    (post['post_PDL1_CPS'] >= 0) & (post['post_PDL1_CPS'] < 1),
    (post['post_PDL1_CPS'] >= 1) & (post['post_PDL1_CPS'] <= 10),
    (post['post_PDL1_CPS'] > 10)
]
choices = [0, 1, 2]
post['encoded_PDL1'] = np.select(conditions, choices, default=np.nan)
post

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,post_wavelet-LLL_glszm_ZoneEntropy,post_wavelet-LLL_glszm_ZonePercentage,post_wavelet-LLL_glszm_ZoneVariance,post_wavelet-LLL_ngtdm_Busyness,post_wavelet-LLL_ngtdm_Coarseness,post_wavelet-LLL_ngtdm_Complexity,post_wavelet-LLL_ngtdm_Contrast,post_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,3.621928,0.003134,1.429334e+06,15.156426,0.003068,0.932926,0.002431,0.037266,10.0,1.0
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,5.709832,0.003814,5.948491e+05,14.851495,0.002102,1.266959,0.005343,0.028783,80.0,2.0
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,4.477880,0.000856,1.198165e+07,74.174093,0.000500,1.435507,0.010100,0.008668,0.0,0.0
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,3.509561,0.001170,1.839469e+07,71.012121,0.000532,1.616311,0.008991,0.006797,10.0,1.0
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,3.628726,0.002132,2.952561e+06,148.316482,0.000469,3.045954,0.013363,0.004109,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,2.404010,0.001197,6.828967e+06,12.033645,0.021085,0.010323,0.000106,0.019846,100.0,2.0
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,4.398636,0.001724,2.190314e+07,92.347019,0.000635,0.615894,0.001000,0.004040,10.0,1.0
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,4.143625,0.000649,6.753192e+07,40.683255,0.000882,0.522524,0.002792,0.009614,50.0,2.0


In [6]:
post.drop(columns=['post_PDL1_CPS'], inplace=True)
post

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,post_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_ZoneEntropy,post_wavelet-LLL_glszm_ZonePercentage,post_wavelet-LLL_glszm_ZoneVariance,post_wavelet-LLL_ngtdm_Busyness,post_wavelet-LLL_ngtdm_Coarseness,post_wavelet-LLL_ngtdm_Complexity,post_wavelet-LLL_ngtdm_Contrast,post_wavelet-LLL_ngtdm_Strength,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,0.035648,3.621928,0.003134,1.429334e+06,15.156426,0.003068,0.932926,0.002431,0.037266,1.0
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,0.012666,5.709832,0.003814,5.948491e+05,14.851495,0.002102,1.266959,0.005343,0.028783,2.0
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,0.200319,4.477880,0.000856,1.198165e+07,74.174093,0.000500,1.435507,0.010100,0.008668,0.0
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,0.013353,3.509561,0.001170,1.839469e+07,71.012121,0.000532,1.616311,0.008991,0.006797,1.0
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,0.043460,3.628726,0.002132,2.952561e+06,148.316482,0.000469,3.045954,0.013363,0.004109,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,0.490065,2.404010,0.001197,6.828967e+06,12.033645,0.021085,0.010323,0.000106,0.019846,2.0
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,0.093450,4.398636,0.001724,2.190314e+07,92.347019,0.000635,0.615894,0.001000,0.004040,1.0
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,0.040322,4.143625,0.000649,6.753192e+07,40.683255,0.000882,0.522524,0.002792,0.009614,2.0


In [7]:
feature = post.drop(columns='encoded_PDL1')
target = post['encoded_PDL1']
list_numerical = feature.columns.to_list()
list_numerical

['post_exponential_firstorder_90Percentile',
 'post_exponential_firstorder_Energy',
 'post_exponential_firstorder_Entropy',
 'post_exponential_firstorder_InterquartileRange',
 'post_exponential_firstorder_Kurtosis',
 'post_exponential_firstorder_Maximum',
 'post_exponential_firstorder_Mean',
 'post_exponential_firstorder_MeanAbsoluteDeviation',
 'post_exponential_firstorder_Median',
 'post_exponential_firstorder_Minimum',
 'post_exponential_firstorder_Range',
 'post_exponential_firstorder_RobustMeanAbsoluteDeviation',
 'post_exponential_firstorder_RootMeanSquared',
 'post_exponential_firstorder_Skewness',
 'post_exponential_firstorder_TotalEnergy',
 'post_exponential_firstorder_Uniformity',
 'post_exponential_firstorder_Variance',
 'post_exponential_glcm_Autocorrelation',
 'post_exponential_glcm_ClusterProminence',
 'post_exponential_glcm_ClusterShade',
 'post_exponential_glcm_ClusterTendency',
 'post_exponential_glcm_Contrast',
 'post_exponential_glcm_Correlation',
 'post_exponential_

In [8]:
feature

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,post_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,post_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_ZoneEntropy,post_wavelet-LLL_glszm_ZonePercentage,post_wavelet-LLL_glszm_ZoneVariance,post_wavelet-LLL_ngtdm_Busyness,post_wavelet-LLL_ngtdm_Coarseness,post_wavelet-LLL_ngtdm_Complexity,post_wavelet-LLL_ngtdm_Contrast,post_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,2.212185,0.035648,3.621928,0.003134,1.429334e+06,15.156426,0.003068,0.932926,0.002431,0.037266
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,2.399476,0.012666,5.709832,0.003814,5.948491e+05,14.851495,0.002102,1.266959,0.005343,0.028783
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,3.027236,0.200319,4.477880,0.000856,1.198165e+07,74.174093,0.000500,1.435507,0.010100,0.008668
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,6.880579,0.013353,3.509561,0.001170,1.839469e+07,71.012121,0.000532,1.616311,0.008991,0.006797
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,5.894270,0.043460,3.628726,0.002132,2.952561e+06,148.316482,0.000469,3.045954,0.013363,0.004109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,0.490065,0.490065,2.404010,0.001197,6.828967e+06,12.033645,0.021085,0.010323,0.000106,0.019846
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,4.318118,0.093450,4.398636,0.001724,2.190314e+07,92.347019,0.000635,0.615894,0.001000,0.004040
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,6.316938,0.040322,4.143625,0.000649,6.753192e+07,40.683255,0.000882,0.522524,0.002792,0.009614


In [9]:
# def custom_zscore(column):
#     if column.std() == 0:
#         return (column - column.mean())  # For columns with zero variance, mean-centered values
#     else:
#         return (column - column.mean()) / column.std()

# # Apply custom_zscore to each column
# feature = feature.apply(custom_zscore)
# feature

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Lasso, LassoCV
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [11]:
# lasso 를 위한 데이터 나누기 
X_train, X_test, y_train, y_test = train_test_split(feature, target, shuffle=True, test_size = 0.2, stratify=target, random_state=2023)

In [12]:
X_train

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,post_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,post_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_ZoneEntropy,post_wavelet-LLL_glszm_ZonePercentage,post_wavelet-LLL_glszm_ZoneVariance,post_wavelet-LLL_ngtdm_Busyness,post_wavelet-LLL_ngtdm_Coarseness,post_wavelet-LLL_ngtdm_Complexity,post_wavelet-LLL_ngtdm_Contrast,post_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,1.627911,24309.456841,-3.203427e-16,0.108932,6.017983,1.938772,1.520828,0.068795,1.523748,0.959160,...,3.712954,0.045867,3.110193,0.009358,9.861540e+05,53.856645,0.002466,0.300491,0.001482,0.005021
37567637,1.483303,28805.828092,3.017787e-03,0.022169,90.191127,2.147837,1.460514,0.016428,1.463005,1.042888,...,2.511693,0.282896,2.500000,0.000593,1.965563e+07,5.052573,0.024931,0.019521,0.000012,0.049891
49087653,1.560215,47505.631970,-3.203427e-16,0.042323,52.445595,1.638994,1.511106,0.034672,1.519543,0.721676,...,2.033580,0.025879,3.121928,0.000482,3.722897e+07,4.640212,0.006835,0.162136,0.000055,0.094512
46695156,1.844094,27042.952113,4.040453e-03,0.156254,6.138386,2.062806,1.648638,0.168702,1.727501,0.751675,...,2.083546,0.014489,3.584963,0.001233,3.078442e+06,20.829426,0.001598,2.261388,0.012098,0.028668
54502130,1.779143,50327.034940,1.160113e-01,0.057897,26.569787,2.702452,1.667244,0.062655,1.659407,0.733516,...,11.578872,0.018783,2.852219,0.001444,6.137747e+06,38.407893,0.000724,3.017443,0.003213,0.015821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,2.583926,53839.454637,9.505661e-01,0.664898,3.372705,3.598932,1.919339,0.380019,1.719894,1.393330,...,0.122289,0.005288,4.247928,0.001373,1.146552e+06,83.509765,0.002002,1.424328,0.016853,0.019469
37509338,1.551646,12486.258508,-3.203427e-16,0.066923,14.469930,1.658007,1.472264,0.069450,1.502731,0.689920,...,0.639736,0.010701,3.000000,0.001398,3.086606e+06,6.305412,0.009324,0.292578,0.000762,0.048651
50740947,1.750991,75489.240704,6.163147e-04,0.041078,37.285483,2.186265,1.695314,0.048156,1.711922,0.778814,...,2.658827,0.015456,3.932138,0.000727,3.048590e+07,10.618826,0.002452,0.500290,0.000542,0.059126


In [13]:
X_test

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,post_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,post_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_ZoneEntropy,post_wavelet-LLL_glszm_ZonePercentage,post_wavelet-LLL_glszm_ZoneVariance,post_wavelet-LLL_ngtdm_Busyness,post_wavelet-LLL_ngtdm_Coarseness,post_wavelet-LLL_ngtdm_Complexity,post_wavelet-LLL_ngtdm_Contrast,post_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
42748515,1.825962,35553.563074,1.503103e-02,0.115020,14.179475,2.199625,1.683764,0.107682,1.718346,0.735869,...,6.444135,0.024791,3.602104,0.004279,1.813493e+06,26.178137,0.001116,2.357668,0.007079,0.025834
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,6.880579,0.013353,3.509561,0.001170,1.839469e+07,71.012121,0.000532,1.616311,0.008991,0.006797
56131813,1.688159,100590.934263,-3.203427e-16,0.053591,37.481021,1.833218,1.631660,0.041272,1.642403,0.734819,...,3.970816,0.093293,4.169982,0.001220,1.440051e+07,236.735525,0.000286,1.674314,0.008835,0.002745
37244770,1.741267,48615.898447,-3.203427e-16,0.046827,42.482157,1.907386,1.684634,0.040553,1.695071,0.822616,...,1.485920,0.044877,3.750000,0.000936,1.509248e+07,15.169934,0.002881,0.364698,0.000638,0.035082
49344356,1.577195,46985.212288,-3.203427e-16,0.058672,5.493569,1.745115,1.449894,0.157792,1.543068,0.758444,...,8.217078,0.020875,3.892865,0.001648,8.946505e+06,26.192858,0.001414,0.846742,0.003784,0.016212
42120720,1.592838,43858.202699,-3.203427e-16,0.060431,6.841173,1.734866,1.465930,0.147654,1.545601,0.725131,...,1.388882,0.006620,4.729283,0.001755,7.959049e+06,22.066897,0.001669,0.799236,0.004507,0.018033
51524302,1.650320,50872.684947,-3.203427e-16,0.037030,50.801961,1.754263,1.612324,0.026148,1.617883,0.943735,...,4.019212,0.070495,3.815541,0.002199,8.255230e+06,14.399798,0.003920,0.233993,0.000115,0.022286
47451087,1.731607,58265.441994,-3.203427e-16,0.034513,45.091798,1.937058,1.692823,0.027326,1.699336,0.909987,...,1.305395,0.288695,3.776618,0.002757,6.545215e+06,34.142405,0.002567,0.154964,0.000771,0.011158
53004114,1.765041,47112.717625,6.428923e-03,0.061759,36.029139,3.408781,1.646975,0.057774,1.647673,0.728600,...,5.444532,0.031945,4.860793,0.003644,1.961045e+06,30.983697,0.000731,3.968143,0.002406,0.021393


In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train[list_numerical]) 

X_train[list_numerical] = scaler.transform(X_train[list_numerical])

X_test[list_numerical] = scaler.transform(X_test[list_numerical])

In [15]:
X_train

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,post_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,post_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_ZoneEntropy,post_wavelet-LLL_glszm_ZonePercentage,post_wavelet-LLL_glszm_ZoneVariance,post_wavelet-LLL_ngtdm_Busyness,post_wavelet-LLL_ngtdm_Coarseness,post_wavelet-LLL_ngtdm_Complexity,post_wavelet-LLL_ngtdm_Contrast,post_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,-0.356290,-0.731212,-0.252203,0.293538,-1.114044,-0.008328,-0.562625,-0.073524,-0.791088,0.594033,...,0.115620,-0.414019,-0.666466,5.176994,-0.782507,-0.047006,-0.135747,-0.688250,-0.583105,-0.295664
37567637,-1.252132,-0.568393,-0.227698,-0.609097,3.852592,0.483972,-1.022962,-0.893341,-1.315827,1.072583,...,-0.368160,1.975759,-1.495927,-0.992723,0.982845,-0.536276,0.057678,-0.973224,-0.836992,0.100585
49087653,-0.775661,0.108748,-0.252203,-0.399425,1.625417,-0.714234,-0.636827,-0.607724,-0.827411,-0.763330,...,-0.560709,-0.615543,-0.650514,-1.070726,2.644547,-0.540410,-0.098127,-0.828576,-0.829620,0.494639
46695156,0.982961,-0.632229,-0.219394,0.785856,-1.106940,0.283744,0.412878,1.490557,0.969062,-0.591864,...,-0.540586,-0.730379,-0.021092,-0.541910,-0.584664,-0.378110,-0.143213,1.300589,1.250459,-0.086835
54502130,0.580590,0.210915,0.689838,-0.237402,0.098615,1.789959,0.554888,-0.169646,0.380824,-0.695654,...,3.283433,-0.687092,-1.017142,-0.393431,-0.295382,-0.201882,-0.150738,2.067417,-0.284215,-0.200286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,5.566203,0.338103,7.466637,6.077577,-1.270129,3.900957,2.478982,4.798796,0.903346,3.075564,...,-1.330436,-0.823145,0.880104,-0.443819,-0.767340,0.250272,-0.139736,0.451601,2.071617,-0.168068
37509338,-0.828746,-1.159343,-0.252203,-0.143504,-0.615337,-0.669462,-0.933286,-0.063273,-0.972647,-0.944829,...,-1.122047,-0.768569,-0.816256,-0.426160,-0.583892,-0.523716,-0.076696,-0.696276,-0.707401,0.089638
50740947,0.406187,1.122067,-0.247199,-0.412377,0.730895,0.574460,0.769127,-0.396627,0.834476,-0.436753,...,-0.308905,-0.720630,0.450838,-0.898499,2.006935,-0.480473,-0.135865,-0.485604,-0.745406,0.182146


In [16]:
n_alphas = 100
alphas = np.logspace(-4, -1.2, n_alphas)
print(alphas)
lasso_cv = LassoCV(alphas = alphas, random_state=2023, max_iter=1000000, cv=KFold(n_splits=5), verbose=True, n_jobs=-1).fit(X_train, y_train)

[0.0001     0.00010673 0.00011391 0.00012158 0.00012976 0.00013849
 0.00014781 0.00015775 0.00016837 0.0001797  0.00019179 0.0002047
 0.00021847 0.00023317 0.00024886 0.00026561 0.00028348 0.00030256
 0.00032292 0.00034465 0.00036784 0.00039259 0.00041901 0.0004472
 0.0004773  0.00050941 0.00054369 0.00058028 0.00061933 0.000661
 0.00070548 0.00075295 0.00080362 0.0008577  0.00091541 0.00097701
 0.00104275 0.00111292 0.00118781 0.00126774 0.00135305 0.0014441
 0.00154127 0.00164498 0.00175568 0.00187382 0.00199991 0.00213448
 0.00227812 0.00243141 0.00259502 0.00276965 0.00295602 0.00315493
 0.00336723 0.00359381 0.00383564 0.00409375 0.00436922 0.00466323
 0.00497702 0.00531193 0.00566938 0.00605088 0.00645804 0.00689261
 0.00735642 0.00785144 0.00837977 0.00894366 0.00954548 0.01018781
 0.01087336 0.01160504 0.01238595 0.01321941 0.01410896 0.01505836
 0.01607166 0.01715313 0.01830738 0.0195393  0.02085412 0.02225742
 0.02375514 0.02535364 0.02705972 0.02888059 0.03082399 0.03289817


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.6s finished


In [17]:
print('R squared training set', round(lasso_cv.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_cv.score(X_test, y_test)*100, 2))

R squared training set 68.22
R squared test set -2.74


In [18]:
best_alpha = lasso_cv.alpha_
print("Best alpha parameter:", best_alpha)

Best alpha parameter: 0.06309573444801933


In [19]:
feature_list = lasso_cv.feature_names_in_[lasso_cv.coef_ != 0]
print(len(feature_list), feature_list)

36 ['post_exponential_glszm_HighGrayLevelZoneEmphasis'
 'post_exponential_glszm_LowGrayLevelZoneEmphasis'
 'post_exponential_glszm_ZonePercentage'
 'post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized'
 'post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance'
 'post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis'
 'post_log-sigma-2-0-mm-3D_glszm_HighGrayLevelZoneEmphasis'
 'post_log-sigma-2-0-mm-3D_glszm_LowGrayLevelZoneEmphasis'
 'post_log-sigma-2-0-mm-3D_ngtdm_Busyness'
 'post_log-sigma-2-5-mm-3D_glcm_MaximumProbability'
 'post_log-sigma-2-5-mm-3D_glszm_SizeZoneNonUniformityNormalized'
 'post_logarithm_gldm_DependenceEntropy'
 'post_logarithm_glrlm_RunVariance'
 'post_logarithm_glszm_GrayLevelVariance'
 'post_original_glszm_LowGrayLevelZoneEmphasis'
 'post_original_shape_Elongation'
 'post_original_shape_Maximum2DDiameterColumn'
 'post_original_shape_Maximum2DDiameterSlice'
 'post_square_glszm_ZoneVariance' 'post_squareroot_glcm_Correlation'
 'post_squareroot_glszm_Gr

In [20]:
lasso_data = feature[feature_list]
lasso_data

,post_exponential_glszm_HighGrayLevelZoneEmphasis,post_exponential_glszm_LowGrayLevelZoneEmphasis,post_exponential_glszm_ZonePercentage,post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized,post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance,post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,post_log-sigma-2-0-mm-3D_glszm_HighGrayLevelZoneEmphasis,post_log-sigma-2-0-mm-3D_glszm_LowGrayLevelZoneEmphasis,post_log-sigma-2-0-mm-3D_ngtdm_Busyness,post_log-sigma-2-5-mm-3D_glcm_MaximumProbability,...,post_wavelet-HLH_glcm_Autocorrelation,post_wavelet-HLH_glcm_MaximumProbability,post_wavelet-HLL_firstorder_Energy,post_wavelet-HLL_firstorder_Minimum,post_wavelet-HLL_firstorder_Range,post_wavelet-HLL_glcm_MaximumProbability,post_wavelet-HLL_glszm_SizeZoneNonUniformityNormalized,post_wavelet-LLH_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_LargeAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.150000,0.962500,0.003134,0.510204,0.244898,1.158957,3.250000,0.437500,534.074369,0.452444,...,2.227702,0.295738,257.784257,-1.564338,3.262799,0.303466,0.333333,0.352105,9.617581e+04,0.120000
17815426,2.750000,0.562500,0.000663,0.509888,0.245056,0.103617,2.981132,0.504717,1850.462552,0.593807,...,2.288415,0.288329,4618.193769,-2.008048,4.857274,0.311828,0.084938,0.255250,4.705706e+04,0.036757
20109646,3.853659,0.286585,0.000835,0.546855,0.226572,0.433379,3.294118,0.426471,7771.438702,0.559628,...,2.242186,0.278136,1530.022754,-2.163784,4.350182,0.315766,0.374486,0.366870,7.347188e+05,0.130385
22320836,1.000000,1.000000,0.000027,0.539776,0.230112,0.727291,3.444444,0.388889,6803.560538,0.708230,...,2.245474,0.276544,1611.882029,-1.928544,3.881885,0.317593,0.217778,0.335456,1.244668e+06,0.138994
32221501,1.000000,1.000000,0.000079,0.654321,0.172840,0.451359,3.666667,0.333333,7560.069705,0.576977,...,2.246201,0.308336,898.908594,-1.935273,4.104258,0.307292,0.250000,0.232143,3.161793e+05,0.215364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.000000,1.000000,0.000109,0.502959,0.248521,0.897160,2.500000,0.625000,1976.791446,0.593749,...,2.253798,0.278207,76.696044,-1.174733,2.429261,0.312252,0.551020,0.345091,1.881904e+06,0.256198
56124435,1.000000,1.000000,0.000022,0.507200,0.246400,0.290226,2.565217,0.608696,7834.861422,0.553779,...,2.246883,0.276533,362.340807,-1.072723,2.149820,0.309319,0.392000,0.419154,2.464875e+06,0.150457
56131318,2.500000,0.625000,0.000033,0.500094,0.249953,0.879929,3.285714,0.428571,105781.893350,0.628251,...,2.234573,0.286447,4785.211884,-1.802331,3.796560,0.313711,0.307755,0.404888,4.527085e+06,0.171598


In [21]:
final_data = pd.concat([lasso_data, target], axis=1)
final_data

,post_exponential_glszm_HighGrayLevelZoneEmphasis,post_exponential_glszm_LowGrayLevelZoneEmphasis,post_exponential_glszm_ZonePercentage,post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized,post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance,post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,post_log-sigma-2-0-mm-3D_glszm_HighGrayLevelZoneEmphasis,post_log-sigma-2-0-mm-3D_glszm_LowGrayLevelZoneEmphasis,post_log-sigma-2-0-mm-3D_ngtdm_Busyness,post_log-sigma-2-5-mm-3D_glcm_MaximumProbability,...,post_wavelet-HLH_glcm_MaximumProbability,post_wavelet-HLL_firstorder_Energy,post_wavelet-HLL_firstorder_Minimum,post_wavelet-HLL_firstorder_Range,post_wavelet-HLL_glcm_MaximumProbability,post_wavelet-HLL_glszm_SizeZoneNonUniformityNormalized,post_wavelet-LLH_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_LargeAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.150000,0.962500,0.003134,0.510204,0.244898,1.158957,3.250000,0.437500,534.074369,0.452444,...,0.295738,257.784257,-1.564338,3.262799,0.303466,0.333333,0.352105,9.617581e+04,0.120000,1.0
17815426,2.750000,0.562500,0.000663,0.509888,0.245056,0.103617,2.981132,0.504717,1850.462552,0.593807,...,0.288329,4618.193769,-2.008048,4.857274,0.311828,0.084938,0.255250,4.705706e+04,0.036757,2.0
20109646,3.853659,0.286585,0.000835,0.546855,0.226572,0.433379,3.294118,0.426471,7771.438702,0.559628,...,0.278136,1530.022754,-2.163784,4.350182,0.315766,0.374486,0.366870,7.347188e+05,0.130385,0.0
22320836,1.000000,1.000000,0.000027,0.539776,0.230112,0.727291,3.444444,0.388889,6803.560538,0.708230,...,0.276544,1611.882029,-1.928544,3.881885,0.317593,0.217778,0.335456,1.244668e+06,0.138994,1.0
32221501,1.000000,1.000000,0.000079,0.654321,0.172840,0.451359,3.666667,0.333333,7560.069705,0.576977,...,0.308336,898.908594,-1.935273,4.104258,0.307292,0.250000,0.232143,3.161793e+05,0.215364,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.000000,1.000000,0.000109,0.502959,0.248521,0.897160,2.500000,0.625000,1976.791446,0.593749,...,0.278207,76.696044,-1.174733,2.429261,0.312252,0.551020,0.345091,1.881904e+06,0.256198,2.0
56124435,1.000000,1.000000,0.000022,0.507200,0.246400,0.290226,2.565217,0.608696,7834.861422,0.553779,...,0.276533,362.340807,-1.072723,2.149820,0.309319,0.392000,0.419154,2.464875e+06,0.150457,1.0
56131318,2.500000,0.625000,0.000033,0.500094,0.249953,0.879929,3.285714,0.428571,105781.893350,0.628251,...,0.286447,4785.211884,-1.802331,3.796560,0.313711,0.307755,0.404888,4.527085e+06,0.171598,2.0


In [22]:
clinical_train = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_train.csv", index_col=0) 
clinical_test = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_test.csv", index_col=0) 
clinical_train.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_test.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_train

,encoded_PDL1
pt_no,
51127213,1.0
38924846,0.0
56241796,1.0
40815882,1.0
33908849,0.0
...,...
47181472,2.0
49087653,1.0
49588132,0.0


In [23]:
rad_features_list = final_data.columns.to_list()
rad_features_list.remove('encoded_PDL1')
rad_features_list

['post_exponential_glszm_HighGrayLevelZoneEmphasis',
 'post_exponential_glszm_LowGrayLevelZoneEmphasis',
 'post_exponential_glszm_ZonePercentage',
 'post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized',
 'post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance',
 'post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis',
 'post_log-sigma-2-0-mm-3D_glszm_HighGrayLevelZoneEmphasis',
 'post_log-sigma-2-0-mm-3D_glszm_LowGrayLevelZoneEmphasis',
 'post_log-sigma-2-0-mm-3D_ngtdm_Busyness',
 'post_log-sigma-2-5-mm-3D_glcm_MaximumProbability',
 'post_log-sigma-2-5-mm-3D_glszm_SizeZoneNonUniformityNormalized',
 'post_logarithm_gldm_DependenceEntropy',
 'post_logarithm_glrlm_RunVariance',
 'post_logarithm_glszm_GrayLevelVariance',
 'post_original_glszm_LowGrayLevelZoneEmphasis',
 'post_original_shape_Elongation',
 'post_original_shape_Maximum2DDiameterColumn',
 'post_original_shape_Maximum2DDiameterSlice',
 'post_square_glszm_ZoneVariance',
 'post_squareroot_glcm_Correlation',
 'post_s

In [24]:
clinical_train[rad_features_list] = final_data[rad_features_list]
clinical_test[rad_features_list] = final_data[rad_features_list]
clinical_train

,encoded_PDL1,post_exponential_glszm_HighGrayLevelZoneEmphasis,post_exponential_glszm_LowGrayLevelZoneEmphasis,post_exponential_glszm_ZonePercentage,post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized,post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance,post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,post_log-sigma-2-0-mm-3D_glszm_HighGrayLevelZoneEmphasis,post_log-sigma-2-0-mm-3D_glszm_LowGrayLevelZoneEmphasis,post_log-sigma-2-0-mm-3D_ngtdm_Busyness,...,post_wavelet-HLH_glcm_Autocorrelation,post_wavelet-HLH_glcm_MaximumProbability,post_wavelet-HLL_firstorder_Energy,post_wavelet-HLL_firstorder_Minimum,post_wavelet-HLL_firstorder_Range,post_wavelet-HLL_glcm_MaximumProbability,post_wavelet-HLL_glszm_SizeZoneNonUniformityNormalized,post_wavelet-LLH_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_LargeAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,3.0,0.5,0.000232,0.563776,0.218112,0.683121,2.578947,0.605263,1433.943739,...,2.274216,0.290826,542.540039,-1.367432,2.699287,0.291946,0.259259,0.323132,2.838964e+05,0.091413
38924846,0.0,1.0,1.0,0.000112,0.882812,0.058594,1.197356,3.750000,0.312500,324.048081,...,2.201245,0.314536,374.582348,-0.861001,3.280302,0.350091,0.277778,0.116283,1.075149e+06,0.148438
56241796,1.0,1.0,1.0,0.000028,0.722222,0.138889,1.197028,3.684211,0.328947,17821.553550,...,2.213589,0.289672,1289.600796,-1.638213,3.233067,0.327116,0.280992,0.207569,1.332111e+06,0.118678
40815882,1.0,1.0,1.0,0.000157,0.722222,0.138889,0.500025,3.500000,0.375000,2413.539474,...,2.242806,0.277992,205.231121,-1.548022,3.535497,0.351960,0.500000,0.327629,2.126472e+05,0.157025
33908849,0.0,3.0,0.5,0.000113,0.543253,0.228374,0.569428,2.636364,0.590909,18470.775010,...,2.231652,0.280518,1106.381188,-2.234428,3.903998,0.330470,0.234568,0.424756,2.870705e+05,0.086207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,1.0,1.0,0.000027,0.516868,0.241566,0.999313,3.000000,0.500000,6429.294709,...,2.249225,0.280753,2221.585695,-1.936985,4.393822,0.308446,0.280000,0.325109,1.288821e+06,0.221607
49087653,1.0,1.0,1.0,0.000048,0.598765,0.200617,1.481504,2.235294,0.691176,16058.062733,...,2.227423,0.292652,1425.845285,-2.204552,5.535875,0.328588,0.285319,0.443038,2.596356e+06,0.120000
49588132,0.0,1.0,1.0,0.000029,0.510204,0.244898,1.311763,3.400000,0.400000,134918.526587,...,2.213065,0.287174,1791.524667,-1.621050,3.647620,0.319112,0.190083,0.390493,7.310622e+05,0.255771


In [25]:
train_data = clinical_train
test_data = clinical_test

In [26]:
# train_data, test_data = train_test_split(final_data, test_size=0.2, shuffle=True, stratify=target, random_state=2021)

In [27]:
train_data

,encoded_PDL1,post_exponential_glszm_HighGrayLevelZoneEmphasis,post_exponential_glszm_LowGrayLevelZoneEmphasis,post_exponential_glszm_ZonePercentage,post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized,post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance,post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,post_log-sigma-2-0-mm-3D_glszm_HighGrayLevelZoneEmphasis,post_log-sigma-2-0-mm-3D_glszm_LowGrayLevelZoneEmphasis,post_log-sigma-2-0-mm-3D_ngtdm_Busyness,...,post_wavelet-HLH_glcm_Autocorrelation,post_wavelet-HLH_glcm_MaximumProbability,post_wavelet-HLL_firstorder_Energy,post_wavelet-HLL_firstorder_Minimum,post_wavelet-HLL_firstorder_Range,post_wavelet-HLL_glcm_MaximumProbability,post_wavelet-HLL_glszm_SizeZoneNonUniformityNormalized,post_wavelet-LLH_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_LargeAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,3.0,0.5,0.000232,0.563776,0.218112,0.683121,2.578947,0.605263,1433.943739,...,2.274216,0.290826,542.540039,-1.367432,2.699287,0.291946,0.259259,0.323132,2.838964e+05,0.091413
38924846,0.0,1.0,1.0,0.000112,0.882812,0.058594,1.197356,3.750000,0.312500,324.048081,...,2.201245,0.314536,374.582348,-0.861001,3.280302,0.350091,0.277778,0.116283,1.075149e+06,0.148438
56241796,1.0,1.0,1.0,0.000028,0.722222,0.138889,1.197028,3.684211,0.328947,17821.553550,...,2.213589,0.289672,1289.600796,-1.638213,3.233067,0.327116,0.280992,0.207569,1.332111e+06,0.118678
40815882,1.0,1.0,1.0,0.000157,0.722222,0.138889,0.500025,3.500000,0.375000,2413.539474,...,2.242806,0.277992,205.231121,-1.548022,3.535497,0.351960,0.500000,0.327629,2.126472e+05,0.157025
33908849,0.0,3.0,0.5,0.000113,0.543253,0.228374,0.569428,2.636364,0.590909,18470.775010,...,2.231652,0.280518,1106.381188,-2.234428,3.903998,0.330470,0.234568,0.424756,2.870705e+05,0.086207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,1.0,1.0,0.000027,0.516868,0.241566,0.999313,3.000000,0.500000,6429.294709,...,2.249225,0.280753,2221.585695,-1.936985,4.393822,0.308446,0.280000,0.325109,1.288821e+06,0.221607
49087653,1.0,1.0,1.0,0.000048,0.598765,0.200617,1.481504,2.235294,0.691176,16058.062733,...,2.227423,0.292652,1425.845285,-2.204552,5.535875,0.328588,0.285319,0.443038,2.596356e+06,0.120000
49588132,0.0,1.0,1.0,0.000029,0.510204,0.244898,1.311763,3.400000,0.400000,134918.526587,...,2.213065,0.287174,1791.524667,-1.621050,3.647620,0.319112,0.190083,0.390493,7.310622e+05,0.255771


In [28]:
print('Data for Modeling: ' + str(train_data.shape))
print('Unseen Data For Predictions: ' + str(test_data.shape))

Data for Modeling: (79, 37)
Unseen Data For Predictions: (22, 37)


In [29]:
ypCR_clf = setup(data=train_data,
            test_data=test_data,
            target='encoded_PDL1',
            normalize=True, 
            normalize_method='zscore', 
            data_split_shuffle=True, 
            data_split_stratify='encoded_PDL1',
            fold = 5,
            fold_strategy = 'stratifiedkfold',
            fold_shuffle = True, 
            session_id=3457, 
            use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

,Description,Value
0,Session id,3457
1,Target,encoded_PDL1
2,Target type,Multiclass
3,Original data shape,"(101, 37)"
4,Transformed data shape,"(101, 37)"
5,Transformed train set shape,"(79, 37)"
6,Transformed test set shape,"(22, 37)"
7,Numeric features,36
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

In [30]:
train = get_config(variable="train_transformed")
train['encoded_PDL1'].value_counts()

1.0    36
0.0    22
2.0    21
Name: encoded_PDL1, dtype: int64

In [31]:
test = get_config(variable="test_transformed")
test['encoded_PDL1'].value_counts()

1.0    10
2.0     6
0.0     6
Name: encoded_PDL1, dtype: int64

In [32]:
compare_models(n_select=3, sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.5950,0.7226,0.5950,0.5827,0.5580,0.3693,0.4039,0.0400
lightgbm,Light Gradient Boosting Machine,0.5200,0.7147,0.5200,0.5183,0.5099,0.2614,0.2643,0.7300
gbc,Gradient Boosting Classifier,0.5417,0.7088,0.5417,0.5919,0.5261,0.2527,0.2749,0.4600
et,Extra Trees Classifier,0.5825,0.7074,0.5825,0.6045,0.5500,0.3315,0.3681,0.2320
rf,Random Forest Classifier,0.4808,0.7012,0.4808,0.5367,0.4595,0.1640,0.1916,0.2640
lda,Linear Discriminant Analysis,0.5833,0.6886,0.5833,0.6106,0.5717,0.3671,0.3843,0.0260
nb,Naive Bayes,0.4700,0.6635,0.4700,0.4098,0.4170,0.2222,0.2388,0.0520
knn,K Neighbors Classifier,0.5575,0.6395,0.5575,0.5579,0.5481,0.3101,0.3175,0.0600
ada,Ada Boost Classifier,0.4942,0.6294,0.4942,0.5885,0.4617,0.1686,0.2041,0.1380
dt,Decision Tree Classifier,0.5050,0.6134,0.5050,0.5388,0.4973,0.2261,0.2357,0.0240


Processing:   0%|          | 0/63 [00:00<?, ?it/s]

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=3457, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                device='gpu', importance_type='split', learning_rate=0.1,
                max_depth=-1, min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
                objective=None, random_state=3457, reg_alpha=0.0, reg_lambda=0.0,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='log_loss', max_depth=3,
                            max_features=No

In [33]:
lightgbm = create_model('lightgbm', fold=5, round=4, cross_validation=True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3125,0.6094,0.3125,0.2964,0.2854,0.0000,0.0000
1,0.6875,0.8112,0.6875,0.6953,0.6875,0.5122,0.5154
2,0.5000,0.7830,0.5000,0.4722,0.4766,0.2099,0.2155
3,0.5000,0.6616,0.5000,0.5278,0.5000,0.2099,0.2155
4,0.6000,0.7083,0.6000,0.6000,0.6000,0.3750,0.3750
Mean,0.5200,0.7147,0.5200,0.5183,0.5099,0.2614,0.2643
Std,0.1251,0.0747,0.1251,0.1338,0.1351,0.1729,0.1732


In [34]:
prediction = predict_model(lightgbm, data=test_data)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.3182,0.5038,0.3182,0.4848,0.3159,-0.0855,-0.0916
